In [1]:
import numpy as np
import cv2
import os
from imutils import paths
from keras.preprocessing.image import img_to_array

import imutils

import matplotlib.pyplot as plt
import PIL
#from PIL import Image

from PIL import Image, ImageOps


# notebook_path = os.path.abspath("Newpest_tensorflow.ipynb")
# #print(notebook_path)

# image_dir = os.path.join(os.path.dirname(notebook_path), "Data/archive/classification/train/KeypointData")
# #Template for Aphids
# aphid24 = os.path.join(os.path.dirname(notebook_path), "Data/archive/templates/template.jpg")

image_dir = "TestData"
#Template for Aphids
aphid = "TestData/template.jpg"



In [2]:
template = cv2.imread(aphid)
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

edge = cv2.Canny(template, 50, 150)
#print(template.shape)
cv2.imshow("Template", template)
cv2.imshow('edge_aphid', edge)

plt.imsave('MatchedImages/edge_aphid.jpg', edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:


tH = template.shape[1]   

tW = template.shape[0]  

template_size = tW, tH
data = []
labels = [] 

imagePaths = np.array(list(paths.list_images(image_dir)))

for (i, imagePath) in enumerate(imagePaths):

    img = cv2.imread(imagePath)
    
    # BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    
##########################################################################################
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    found = None
# loop over the scales of the image
    for scale in np.linspace(0.2, 1.0, 20)[::-1]:
# resize the image according to the scale, and keep track
# of the ratio of the resizing
        resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
        r = gray.shape[1] / float(resized.shape[1])
# if the resized image is smaller than the template, then break
# from the loop
        if resized.shape[0] < tH or resized.shape[1] < tW:
            #continue
            print('size different')
        break
            
# matching to find the template in the image
    edged = cv2.Canny(img, 50, 200)
    result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
    (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

# bounding box around the detected region
    clone = np.dstack([edged, edged, edged])
    cv2.rectangle(clone, (maxLoc[0], maxLoc[1]),
    (maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
    #cv2.imshow("Visualize", clone)
    #cv2.waitKey(0)
    
    if found is None or maxVal > found[0]:
        found = (maxVal, maxLoc, r)
# unpack the bookkeeping variable and compute the (x, y) coordinates
# of the bounding box based on the resized ratio
        (_, maxLoc, r) = found
        (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
        (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
        
# draw a bounding box around the detected result and display the image
        filename = 'MatchedImages/matched_'+ str(i)+'.jpg'
        cv2.rectangle(img, (startX, startY), (endX, endY), (0, 0, 255), 2)
        #cv2.imshow("Image", img)
        plt.imsave(filename, img)
        #cv2.waitKey(0)        
        

size different
